# Value proposition of norfair

Norfair is a customizable lightweight Python library for real-time multi-object tracking.
Using Norfair, you can add tracking capabilities to any detector with just a few lines of code.

It means you won't need a SOTA Tracker you can use a basic Tracker with a Kalmann Filter and add the custom logic you want.

# Imports and setup

In [ ]:
import sys; sys.path.append('.')
import os

import cv2
from norfair import Tracker, OptimizedKalmanFilterFactory

from lib.sequence import Sequence
from lib.norfair_helper.video import generate_tracking_video


If you want to test this code on your detection and frames you can use the following code if you structure the data as follows:

```
data/
   ├── detection/
   │   └── sequence_1/
   │       └── detections_1.txt
   └── frames/
       └── sequence_1/
           └── frame_1.jpg
```

Where the detections.txt file is in the following format scaled between 0 and 1:

```
class_id x_center y_center width height confidence
```

If this is not the case, you'll need to adapt this code to your data.

In [ ]:
DATA_PATH = "../data"
DETECTION_PATH = f"{DATA_PATH}/detections"
FRAME_PATH = f"{DATA_PATH}/frames"
VIDEO_OUTPUT_PATH = "private"

SEQUENCES = os.listdir(FRAME_PATH)

In [ ]:
def get_sequence_frames(sequence):
    frames = os.listdir(f"{FRAME_PATH}/{sequence}")
    frames = [os.path.join(f"{FRAME_PATH}/{sequence}", frame) for frame in frames]
    frames.sort()
    return frames

def get_sequence_detections(sequence):
    detections = os.listdir(f"{DETECTION_PATH}/{sequence}")
    detections = [os.path.join(f"{DETECTION_PATH}/{sequence}", detection) for detection in detections]
    detections.sort()
    return detections

frame_path = get_sequence_frames(SEQUENCES[3])
test_sequence = Sequence(frame_path)
test_sequence

In [ ]:
test_sequence

# Basic Usage of Norfair

## Tracker

Norfair tracker object is the customizable object that will track detections.
Norfair expects a distance function that will serve as a metric to match objects between each detection. You can create your own distance metric or use one of the built-in ones such as euclidian distance, iou or many more.

In [ ]:
# Initialize a tracker with the distance function
basic_tracker = Tracker(
    distance_function="mean_euclidean",
    distance_threshold=40,
)

## Basic tracking

In [ ]:
video_path = generate_tracking_video(
    sequence=test_sequence,
    tracker=basic_tracker,
    frame_size=(2560, 1440),
    output_path=os.path.join(VIDEO_OUTPUT_PATH, "basic_tracking.mp4"),
    add_embedding=False,
)
video_path

## Advanced tracking

In [ ]:
def always_match(new_object, unmatched_object):
    return 0 # ALWAYS MATCH


def embedding_distance(matched_not_init_trackers, unmatched_trackers):
    snd_embedding = unmatched_trackers.last_detection.embedding

    # Find last non-empty embedding if current is None
    if snd_embedding is None:
        snd_embedding = next((detection.embedding for detection in reversed(unmatched_trackers.past_detections) if detection.embedding is not None), None)

    if snd_embedding is None:
        return 1 # No match if no embedding is found

    # Iterate over past detections and calculate distance
    for detection_fst in matched_not_init_trackers.past_detections:
        if detection_fst.embedding is not None:
            distance = 1 - cv2.compareHist(snd_embedding, detection_fst.embedding, cv2.HISTCMP_CORREL)
            # If similar a tiny bit similar, we return the distance to the tracker
            if distance < 0.9:
                return distance

    return 1 # No match if no matching embedding is found between the 2

In [ ]:
advanced_tracker = Tracker(
    distance_function="sqeuclidean",
    filter_factory = OptimizedKalmanFilterFactory(R=5, Q=0.05),
    distance_threshold=350, # Higher value means objects further away will be matched
    initialization_delay=12, # Wait 15 frames before an object is starts to be tracked
    hit_counter_max=15, # Inertia, higher values means an object will take time to enter in reid phase
    reid_distance_function=embedding_distance, # function to decide on which metric to reid
    reid_distance_threshold=0.9, # If the distance is below the object is matched
    reid_hit_counter_max=200, #higher values means an object will stay reid phase longer
    )

In [ ]:
video_path = generate_tracking_video(
    sequence=test_sequence,
    tracker=advanced_tracker,
    frame_size=(2560, 1440),
    output_path=os.path.join(VIDEO_OUTPUT_PATH, "advance_tracking.mp4"),
    add_embedding=True,
)
video_path

In [ ]:
advanced_tracker.total_object_count